In [ ]:
# Recurrent Neural Network - Train
간단하게 FCNN 과 CNN 을 설명하였다. RNN 은 Recurrent Neural Network 의 약자로 순차적인 데이터를 처리하기 위하여 고안이 되었다.  
예를들면 자연어 데이터 같은 것이 있을 수 있다. 본 예제는 https://github.com/sjchoi86/Tensorflow-101 을 참조하여 작성되었다. 

<H3>(1) Package Load</H3>   
아무래도 RNN 은 데이터 전처리 작업이 많다보니 여러가지 Package 들을 사용하게 된다. 

In [1]:
# Import Packages
import numpy as np
import tensorflow as tf
import collections
import argparse
import time
import os
from six.moves import cPickle
print ("Packages Imported")

Packages Imported


<H3>(3) Data Load</H3>   
데이터가 잘 Load 되었는지 확인한다. Sample Data 는 C 프로그램으로 보인다. 

In [2]:
# Load text
# data_dir    = "data/tinyshakespeare"
data_dir    = "RNN_data/"
save_dir    = "RNN_data/"
#tensorboard log directoy
logs_path = '/tmp/tensorflow_logs/chap4-rnn-eng'
input_file  = os.path.join(data_dir, "input.txt")
with open(input_file, "r") as f:
    data = f.read()
print("=============load file")
print("Text loaded from '%s'" % (input_file))
print("=============file contents")
print("Text Length : {0}".format(data[4000:4500]))
print("=============file length")
print("Text Length : {0}".format(len(data)))

=============load file
Text loaded from 'RNN_data/input.txt'
=============file contents
Text Length : * SUSPEND;
	resume = sbuf.f_blocks * RESUME;

	sector_div(suspend, 100);
	sector_div(resume, 100);

	if (sbuf.f_bavail <= suspend)
		act = -1;
	else if (sbuf.f_bavail >= resume)
		act = 1;
	else
		act = 0;

	/*
	 * If some joker switched acct_globals.file under us we'ld better be
	 * silent and _not_ touch anything.
	 */
	spin_lock(&acct_globals.lock);
	if (file != acct_globals.file) {
		if (act)
			res = act>0;
		goto out;
	}

	if (acct_globals.active) {
		if (act < 0) {
			acct_globals.active 
=============file length
Text Length : 1708864


<H3>(4) Distinct, Sort</H3>   
Contents 에 사용된 Distinct 한 단어와 Count 횟수를 Pair 하게 Mapping 하고 Count 역순으로 정렬한다. 

In [3]:
# Preprocess Text
# First, count the number of characters
counter = collections.Counter(data)
count_pairs = sorted(counter.items(), key=lambda x: -x[1]) # <= Sort

print ("Type of 'counter.items()' is %s and length is %d" 
       % (type(counter.items()), len(counter.items()))) 
for i in range(5):
    print ("[%d/%d]" % (i, 3)), # <= This comma remove '\n'
    print (list(counter.items())[i])

print (" ")
print ("Type of 'count_pairs' is %s and length is %d" 
       % (type(count_pairs), len(count_pairs))) 
for i in range(5):
    print ("[%d/%d]" % (i, 3)), # <= This comma remove '\n'
    print (count_pairs[i])

Type of 'counter.items()' is <class 'dict_items'> and length is 98
[0/3]
('1', 2444)
[1/3]
('\\', 836)
[2/3]
('@', 677)
[3/3]
(' ', 171219)
[4/3]
('$', 61)
 
Type of 'count_pairs' is <class 'list'> and length is 98
[0/3]
(' ', 171219)
[1/3]
('e', 113021)
[2/3]
('t', 102154)
[3/3]
('r', 76185)
[4/3]
('i', 75487)


<H3>(5) vocab & chars</H3>   
vacab 을 사용하여 Char 의 Index 를 구할 수 있으며, chars 를 사용하여 Index 로 Char 를 구할 수 있다.  
Voca Type 과 Voca Contents 로 데이터를 구성하여 voca file 을 만들어 저장한다.  

In [4]:
# Let's make dictionary
chars, counts = zip(*count_pairs)

vocab = dict(zip(chars, range(len(chars))))

print ("Type of 'chars' is %s and length is %d" 
    % (type(chars), len(chars))) 
for i in range(5):
    print ("[%d/%d]" % (i, 3)), # <= This comma remove '\n'
    print ("chars[%d] is '%s'" % (i, chars[i]))
    
print ("")

print ("Type of 'vocab' is %s and length is %d" 
    % (type(vocab), len(vocab))) 
for i in range(5):
    print ("[%d/%d]" % (i, 3)), # <= This comma remove '\n'
    print ("vocab['%s'] is %s" % (chars[i], vocab[chars[i]]))
    
# SAve chars and vocab
with open(os.path.join(save_dir, 'chars_vocab.pkl'), 'wb') as f:
    cPickle.dump((chars, vocab), f)

Type of 'chars' is <class 'tuple'> and length is 98
[0/3]
chars[0] is ' '
[1/3]
chars[1] is 'e'
[2/3]
chars[2] is 't'
[3/3]
chars[3] is 'r'
[4/3]
chars[4] is 'i'

Type of 'vocab' is <class 'dict'> and length is 98
[0/3]
vocab[' '] is 0
[1/3]
vocab['e'] is 1
[2/3]
vocab['t'] is 2
[3/3]
vocab['r'] is 3
[4/3]
vocab['i'] is 4


<H3>(6) Corpus</H3>   
위에서 생성한 vocab (char : 순번) 형태의 Dict 데이터를 활용하여 훈련하고자 하는 전문을 [ 8 14 40 36 24  1  3  8  1 13] 형태로 바꾼다 로그를 보면 원하는 대로 잘 변환이 된 것을 확일 할 수 있다. 

In [5]:
# Now convert all text to index using vocab! 
corpus = np.array(list(map(vocab.get, data)))
print ("Type of 'corpus' is %s, shape is %s, and length is %d" 
    % (type(corpus), corpus.shape, len(corpus)))

check_len = 10
print ("\n'corpus' looks like %s" % (corpus[0:check_len]))
for i in range(check_len):
    _wordidx = corpus[i]
    print ("[%d/%d] chars[%02d] corresponds to '%s'" 
           % (i, check_len, _wordidx, chars[_wordidx]))

Type of 'corpus' is <class 'numpy.ndarray'>, shape is (1708864,), and length is 1708864

'corpus' looks like [ 8 14 40 36 24  1  3  8  1 13]
[0/10] chars[08] corresponds to 'n'
[1/10] chars[14] corresponds to 'u'
[2/10] chars[40] corresponds to 'x'
[3/10] chars[36] corresponds to '/'
[4/10] chars[24] corresponds to 'k'
[5/10] chars[01] corresponds to 'e'
[6/10] chars[03] corresponds to 'r'
[7/10] chars[08] corresponds to 'n'
[8/10] chars[01] corresponds to 'e'
[9/10] chars[13] corresponds to 'l'


<H3>(7) Prepare Train Data</H3>   
Encode 200 , Decode 200 인 Sequence2Sequece Model 을 위한 Data 를 구성한다. 원래 데이터의 길이는 1708864 인데 자투리는 정리하고 1700000 사이즈로 만들어 주고 XData와  YData 를 구성한다. Ydata (Decode) 데이터는 XData 와 같은 데이터를 사용하되 한 칸씩 데이터를 밀어준다.   
XData가 [a,b,c,d,e] 라면 YData 는 [b,c,d,e,f] 식으로 말이다.  
최종적으로 구성되는 데이터는 각 XData 와 YData 에 대하여 200 X 50 Matrix 가 170개 있는 형태로 구성이 될 것이다.  

In [6]:
# Generate batch data 
batch_size  = 50
seq_length  = 200
num_batches = int(corpus.size / (batch_size * seq_length))
# First, reduce the length of corpus to fit batch_size
corpus_reduced = corpus[:(num_batches*batch_size*seq_length)]
xdata = corpus_reduced
ydata = np.copy(xdata)
ydata[:-1] = xdata[1:]
ydata[-1]  = xdata[0]

print ('xdata is ... %s and length is %d' % (xdata, xdata.size))
print ('ydata is ... %s and length is %d' % (ydata, xdata.size))
print ("")

# Second, make batch 
xbatches = np.split(xdata.reshape(batch_size, -1), num_batches, 1)
ybatches = np.split(ydata.reshape(batch_size, -1), num_batches, 1)
print ("Type of 'xbatches' is %s and length is %d" 
    % (type(xbatches), len(xbatches)))
print ("Type of 'ybatches' is %s and length is %d" 
    % (type(ybatches), len(ybatches)))
print(xbatches[169])
print ("")


# How can we access to xbatches?? 
nbatch = 5
temp = xbatches[0:nbatch]
print ("Type of 'temp' is %s and length is %d" 
    % (type(temp), len(temp)))
for i in range(nbatch):
    temp2 = temp[i]
    print ("Type of 'temp[%d]' is %s and shape is %s" % (i, type(temp2), temp2.shape,))

xdata is ... [ 8 14 40 ...,  3  1  1] and length is 1700000
ydata is ... [14 40 36 ...,  1  1  8] and length is 1700000

Type of 'xbatches' is <class 'list'> and length is 170
Type of 'ybatches' is <class 'list'> and length is 170
[[16 26  0 ..., 10 13 11]
 [ 3  4  8 ..., 13  4 19]
 [18 20 23 ..., 24 21 46]
 ..., 
 [40 10 15 ...,  4  2  1]
 [ 7  6 12 ...,  8 13  4]
 [ 4  5  2 ...,  3  1  1]]

Type of 'temp' is <class 'list'> and length is 5
Type of 'temp[0]' is <class 'numpy.ndarray'> and shape is (50, 200)
Type of 'temp[1]' is <class 'numpy.ndarray'> and shape is (50, 200)
Type of 'temp[2]' is <class 'numpy.ndarray'> and shape is (50, 200)
Type of 'temp[3]' is <class 'numpy.ndarray'> and shape is (50, 200)
Type of 'temp[4]' is <class 'numpy.ndarray'> and shape is (50, 200)


<H3>(8) Net Config </H3>   
RNN 은 전처리가 많다보니 이제.. net config 부분이다..  RNN 은 크게 Vanlia, LSTM, GRU 정도로 나눌 수 있는데, Tensorflow 에서 구현시에는 함수 명만 달라지는 정도로 나머지는 Tensorflow 가 알아서 해준다.. Vanila 의 경우 간단한 예제에서만 사용하는 정도로 Vanishing Problem 때문에 사용하지 않는다.. 여기서는 LSTM 을 사용하는 예제로 모델 구성은 그림을 그리면 간단할태지만 말로 설명하자면 각 Cell 은 LSTM 으로 구성되어 있고 Depth는 2 , 그리고 Cell 의 Size 는 128 로 구성되어 있으며, Encode 부와 Decode 부는 각 200인 형태의 RNN 모델이 되겠다.  

In [7]:
# Important RNN parameters 
vocab_size = len(vocab)
rnn_size   = 128
num_layers = 2
grad_clip  = 5.


# Construct RNN model 
unitcell   = tf.contrib.rnn.BasicLSTMCell(rnn_size)
cell       = tf.contrib.rnn.MultiRNNCell([unitcell] * num_layers)
input_data = tf.placeholder(tf.int32, [batch_size, seq_length])
targets    = tf.placeholder(tf.int32, [batch_size, seq_length])
istate     = cell.zero_state(batch_size, tf.float32)
# Weigths 
with tf.variable_scope('rnnlm'):
    softmax_w = tf.get_variable("softmax_w", [rnn_size, vocab_size])
    softmax_b = tf.get_variable("softmax_b", [vocab_size])
    with tf.device("/cpu:0"):
        embedding = tf.get_variable("embedding", [vocab_size, rnn_size])
        inputs = tf.split(tf.nn.embedding_lookup(embedding, input_data), seq_length, 1)
        inputs = [tf.squeeze(_input, [1]) for _input in inputs]
# Output
def loop(prev, _):
    prev = tf.nn.xw_plus_b(prev, softmax_w, softmax_b)
    prev_symbol = tf.stop_gradient(tf.argmax(prev, 1))
    return tf.nn.embedding_lookup(embedding, prev_symbol)
"""
    loop_function: If not None, this function will be applied to the i-th output
    in order to generate the i+1-st input, and decoder_inputs will be ignored,
    except for the first element ("GO" symbol).
""" 
outputs, last_state = tf.contrib.legacy_seq2seq.rnn_decoder(inputs, istate, cell
                , loop_function=None, scope='rnnlm')

# print("outputs : {0}".format(outputs))
print("concat : {0}".format(tf.concat(outputs, 1)))
print("rnn_size : {0}".format(rnn_size))
print("reshape : {0}".format(tf.reshape(tf.concat(outputs, 1), [-1, rnn_size])))
output = tf.reshape(tf.concat(outputs, 1), [-1, rnn_size])
logits = tf.nn.xw_plus_b(output, softmax_w, softmax_b)
probs  = tf.nn.softmax(logits)
# Loss
loss = tf.contrib.legacy_seq2seq.sequence_loss_by_example([logits], # Input
    [tf.reshape(targets, [-1])], # Target
    [tf.ones([batch_size * seq_length])], # Weight 
    vocab_size)
# Optimizer
cost     = tf.reduce_sum(loss) / batch_size / seq_length
final_state = last_state
lr       = tf.Variable(0.0, trainable=False)
tvars    = tf.trainable_variables()
grads, _ = tf.clip_by_global_norm(tf.gradients(cost, tvars), grad_clip)
_optm    = tf.train.AdamOptimizer(lr)
optm     = _optm.apply_gradients(zip(grads, tvars))

print ("Network Ready")

concat : Tensor("concat:0", shape=(50, 25600), dtype=float32)
rnn_size : 128
reshape : Tensor("Reshape:0", shape=(10000, 128), dtype=float32)
Network Ready


<H3>(9) Session 실행 </H3>   
갱신된 State 를 받아서 다시 입력으로 State 를 넣어 주는 부분을 제외하고는 특이한 부분은 없다. 

In [8]:
# Train the model!
num_epochs    = 50
save_every    = 500
learning_rate = 0.002
decay_rate    = 0.97

sess = tf.Session()
sess.run(tf.initialize_all_variables())
tf.summary.FileWriter(logs_path, graph=tf.get_default_graph())
saver = tf.train.Saver(tf.all_variables())
init_time = time.time()
for epoch in range(num_epochs):
    # Learning rate scheduling 
    sess.run(tf.assign(lr, learning_rate * (decay_rate ** epoch)))
    state     = sess.run(istate)
    batchidx  = 0
    for iteration in range(num_batches):
        start_time   = time.time()
        randbatchidx = np.random.randint(num_batches)
        xbatch       = xbatches[batchidx]
        ybatch       = ybatches[batchidx]
        batchidx     = batchidx + 1
        
        # Note that, num_batches = len(xbatches)
        # Train! 
        train_loss, state, _ = sess.run([cost, final_state, optm]
            , feed_dict={input_data: xbatch, targets: ybatch, istate: state}) 
        total_iter = epoch*num_batches + iteration
        end_time     = time.time();
        duration     = end_time - start_time
        
        if total_iter % 100 == 0:
            print("==== xbatch : {0}".format(xbatch))
            print("==== ybatch : {0}".format(ybatch))
            print ("[%d/%d] cost: %.4f / Each batch learning took %.4f sec" 
                   % (total_iter, num_epochs*num_batches, train_loss, duration))
        if total_iter % save_every == 0: 
            ckpt_path = os.path.join(save_dir, 'model.ckpt')
            saver.save(sess, ckpt_path, global_step = total_iter)
            # Save network! 
            print("model saved to '%s'" % (ckpt_path))

Instructions for updating:
Use `tf.global_variables_initializer` instead.
Instructions for updating:
Please use tf.global_variables instead.
==== xbatch : [[ 8 14 40 ...,  1  3  5]
 [12 24 21 ..., 27  9  3]
 [ 4  2  0 ...,  2 21 52]
 ..., 
 [49 48 50 ..., 20 23  7]
 [ 3 21 32 ...,  1  0 15]
 [ 8  1 10 ...,  8  0 38]]
==== ybatch : [[14 40 36 ...,  3  5 35]
 [24 21 46 ...,  9  3  3]
 [ 2  0  1 ..., 21 52 11]
 ..., 
 [48 50 10 ..., 23  7  6]
 [21 32 20 ...,  0 15  1]
 [ 1 10 19 ...,  0 38 47]]
[0/8500] cost: 4.5940 / Each batch learning took 2.7463 sec
model saved to 'RNN_data/model.ckpt'
==== xbatch : [[ 8 13  4 ...,  0 22 36]
 [38 53 45 ..., 33 45 26]
 [ 6  4 17 ..., 10 50 33]
 ..., 
 [ 9  4  2 ...,  0 12 14]
 [ 3  3 23 ..., 29  0 30]
 [ 2  5  9 ..., 14 16  2]]
==== ybatch : [[13  4  8 ..., 22 36  7]
 [53 45 45 ..., 45 26  0]
 [ 4 17  0 ..., 50 33 41]
 ..., 
 [ 4  2  1 ..., 12 14  3]
 [ 3 23  7 ...,  0 30  4]
 [ 5  9 12 ..., 16  2  4]]
[100/8500] cost: 3.1131 / Each batch learning took